In [1]:
import keras
import tensorflow as tf
import numpy as np
from keras.datasets import mnist
import pandas as pd
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Layer
from ode_block import ODEBlock2D, ODEWrapper

Using TensorFlow backend.


In [7]:
from keras import backend as K
class ResBlock(Layer):

    def __init__(self, filters, kernel_size, **kwargs):
        self.filters = filters
        self.kernel_size = kernel_size
        super(ResBlock, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.conv2d_w1 = self.add_weight("conv2d_w1", self.kernel_size + (self.filters, self.filters), initializer='glorot_uniform')
        self.conv2d_w2 = self.add_weight("conv2d_w2", self.kernel_size + (self.filters, self.filters), initializer='glorot_uniform')
        self.conv2d_b1 = self.add_weight("conv2d_b1", (self.filters,), initializer='zero')
        self.conv2d_b2 = self.add_weight("conv2d_b2", (self.filters,), initializer='zero')
        super(ResBlock, self).build(input_shape)

    def call(self, x):
        y = K.conv2d(x, self.conv2d_w1, padding="same")
        y = K.bias_add(y, self.conv2d_b1)
        y = K.relu(y)

        y = K.conv2d(y, self.conv2d_w2, padding="same")
        y = K.bias_add(y, self.conv2d_b2)
        y = K.relu(y)

        y = y+x
        return y

    def compute_output_shape(self, input_shape):
        return input_shape

In [3]:
batch_size = 128
num_classes = 10
epochs = 10
image_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape((-1,) + image_shape)
x_test = x_test.reshape((-1,) + image_shape)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)


In [8]:
def build_model(input_shape, num_classes):
    x = Input(input_shape)
    y = Conv2D(32, (3, 3), activation='relu')(x)
    y = MaxPooling2D((2,2))(y)
    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D((2,2))(y)
    y = ODEBlock2D(64, (3, 3))(y)
    y = Flatten()(y)
    y = Dense(num_classes, activation='softmax')(y)
    return Model(x,y)

In [9]:
def build_res_model(input_shape, num_classes):
    x = Input(input_shape)
    y = Conv2D(32, (3, 3), activation='relu')(x)
    y = MaxPooling2D((2,2))(y)
    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D((2,2))(y)
    y = ResBlock(64, (3, 3))(y)
    y = Flatten()(y)
    y = Dense(num_classes, activation='softmax')(y)
    return Model(x,y)

In [10]:
model_res = build_res_model(image_shape, num_classes)

model_res.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])



In [11]:
model_res.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
res_block_2 (ResBlock)       (None, 5, 5, 64)          73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
__________

In [12]:
model_res.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


score = model_res.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 49s 809us/step - loss: 0.2119 - acc: 0.9353 - val_loss: 0.0485 - val_acc: 0.9834
Epoch 2/10
60000/60000 [==============================] - 45s 742us/step - loss: 0.0523 - acc: 0.9841 - val_loss: 0.0456 - val_acc: 0.9853
Epoch 3/10
60000/60000 [==============================] - 44s 741us/step - loss: 0.0350 - acc: 0.9892 - val_loss: 0.0269 - val_acc: 0.9913
Epoch 4/10
60000/60000 [==============================] - 45s 742us/step - loss: 0.0263 - acc: 0.9913 - val_loss: 0.0280 - val_acc: 0.9907
Epoch 5/10
60000/60000 [==============================] - 44s 741us/step - loss: 0.0213 - acc: 0.9930 - val_loss: 0.0316 - val_acc: 0.9896
Epoch 6/10
60000/60000 [==============================] - 44s 738us/step - loss: 0.0190 - acc: 0.9939 - val_loss: 0.0262 - val_acc: 0.9924
Epoch 7/10
60000/60000 [==============================] - 44s 740us/

In [13]:
model = build_model(image_shape, num_classes)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
ode_block2d_1 (ODEBlock2D)   (None, 5, 5, 64)          75008     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
__________

In [15]:
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    K.set_session(tf.Session(config=config))

set_gpu_config("0",0.25)

In [16]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 163s 3ms/step - loss: 0.2209 - acc: 0.9334 - val_loss: 0.0503 - val_acc: 0.9852
Epoch 2/10
60000/60000 [==============================] - 153s 3ms/step - loss: 0.0516 - acc: 0.9844 - val_loss: 0.0383 - val_acc: 0.9867
Epoch 3/10
60000/60000 [==============================] - 153s 3ms/step - loss: 0.0346 - acc: 0.9894 - val_loss: 0.0295 - val_acc: 0.9907
Epoch 4/10
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0270 - acc: 0.9914 - val_loss: 0.0260 - val_acc: 0.9912
Epoch 5/10
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0223 - acc: 0.9929 - val_loss: 0.0332 - val_acc: 0.9892
Epoch 6/10
60000/60000 [==============================] - 153s 3ms/step - loss: 0.0193 - acc: 0.9939 - val_loss: 0.0285 - val_acc: 0.9907
Epoch 7/10
60000/60000 [==============================] - 154s 3ms/step - loss: 0.0150 - acc: 0.9952 - val_loss: 0.0234 - 